<a href="https://colab.research.google.com/github/2201018/OfficeWorkManual/blob/main/SD4_HB_Section09_01_13_%E5%B1%80%E5%84%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 高度プログラミング演習Ｂ プログラムテンプレート
## 統一指示


1.   以下の形式でファイル名の「出席番号_フルネーム」の箇所を自分にあわせて変更してください
2.   メニューの[ファイル]->[ドライブにコピーを保存]から自分のGoogleDrive内にコピーを保存してから、編集してください。最後に指示があったら、メニューの[ファイル]->[Githubにコピーを保存]からGithubOrganizationの25SD4組織内にコピーを保存・提出してください。





In [1]:
!pip install -U -q langgraph langchain-core langchain-google-genai python-dotenv pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
gr

In [2]:
import os
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
print("APIキーの設定完了")

APIキーの設定完了


In [3]:
import operator
from typing import Annotated, List, Any
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph

In [4]:
class State(BaseModel):
  query:str = Field(  description ="ユーザーからの質問")
  current_role:str = Field(default="",description="選定された回答ロール")
  massages:Annotated[list[str], operator.add] = Field(default=[],description="回答履歴")
  current_judge:bool=Field(default=False,description="品質チェックの結果")
  judgement_reson:str = Field(default="",description="品質チェックの判定理由")


In [5]:
workflow = StateGraph(State)
print("グラフのワークフローのインスタンスが生成されました")

グラフのワークフローのインスタンスが生成されました


In [6]:
def selection_node(state:State) -> dict[str,Any]:
  print("---喉　: selection(ロール選定の実行)---")
  selection_role="生成AI製品エキスパート"
  print(f"選定ロール:{selected_role}")
  return {"curent_tole":selected_role}

In [7]:
def check_node(state:State) -> dict[str,Any]:
  print("---ノード：check（品質チェック）実行---")
  query = state.query
  message = state.messages[-1]
  judge = True
  reason = "ダミーの判定理由:問題なし"
  print(f" 判定結果:{judge},理由:{reason}")
  return{"curente_judge":judge,
         "judgement_reason":reason}


In [8]:
workflow.add_node("selection",selection_node)
workflow.add_node("check",check_node)

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

In [10]:
prompt = ChatPromptTemplate.from_template(
    "{query}(ロール:{role})"
    )
llm = ChatGoogleGenerativeAI(
    model="gemin-flash-latest",
    temperature = 0,
    api_key = GEMINI_API_KEY
)

In [22]:
answering_node = (
    RunnableLambda(
        lambda state:{
            "query":state.query,
            "role":state.cuerrent_role
        }
    |prompt
    |llm
    |StrOutputParser()
    |RunnableLambda(lambda x:{"message":[x]})
)
)

In [23]:
workflow.add_node("answering",answering_node)

In [13]:
workflow.set_entry_point("selection")

In [14]:
workflow.add_edge("selection","answering")
workflow.add_edge("answering","check")

In [15]:
from langgraph.graph import END
workflow.add_conditional_edges(
    "check",
    lambda state:state.current_judge,
    {True:END,
     False:"selection",
    }
)

In [16]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [24]:
compiled_graph = workflow.compile()

In [25]:
import pprint
print("--- 1: invoke(同期実行)---")
initial_state_invoke = {
    "query":"生成AIの概要と利用状況と問題点について詳細に解説してください。"
}
final_result_invoke = compiled_graph.invoke
(initial_state_invoke)
pprint.pprint(final_result_invoke)

--- 1: invoke(同期実行)---
<bound method Pregel.invoke of <langgraph.graph.state.CompiledStateGraph object at 0x7a9457380530>>


In [26]:
print("--- 1: invoke(非同期実行)---")
initial_state_ainvoke = {
    "query":"Langgraphの概要と利用状況と問題点について詳細に解説してください"
}
final_result_ainvoke = await compiled_graph.ainvoke(initial_state_ainvoke)
pprint.pprint(final_result_ainvoke)

--- 1: invoke(非同期実行)---
---喉　: selection(ロール選定の実行)---


NameError: name 'selected_role' is not defined

In [27]:
print("---3:stream---")
initial_state_stream = {
    "query":"LangGraphのStateの概要と利用状況と問題点について詳細に解説してください。"
}

for step in compiled_graph.stream(initial_state_stream):
  print(step)
  print("---")

---3:stream---
---喉　: selection(ロール選定の実行)---


NameError: name 'selected_role' is not defined